# MLPY: Getting Started (Actualizado 2025)

Este notebook te introducirá a los conceptos básicos de MLPY, un framework moderno de machine learning para Python.

## 1. Instalación y Setup

Primero, asegúrate de tener MLPY instalado:

In [ ]:
# Si no está instalado, descomenta la siguiente línea:
# !pip install mlpy

import mlpy
print(f"MLPY version: {mlpy.__version__}")
print(f"MLPY está 100% funcional desde agosto 2025")

## 2. Conceptos Básicos

MLPY está construido alrededor de varios conceptos clave:

- **Task**: Encapsula los datos y metadatos de un problema de ML
- **Learner**: Interfaz unificada para algoritmos de ML
- **Measure**: Métricas para evaluar el rendimiento
- **Resampling**: Estrategias para evaluación robusta

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris

# Importar componentes principales de MLPY
from mlpy.tasks import TaskClassif
from mlpy.learners import LearnerClassifSklearn
from mlpy.measures import MeasureClassifAccuracy, MeasureClassifF1
from mlpy.resamplings import ResamplingCV
from mlpy import resample

## 3. Crear una Tarea (Task)

Vamos a trabajar con el dataset Iris:

In [ ]:
# Cargar datos
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['species'] = iris.target_names[iris.target]

print(f"Dataset shape: {df.shape}")
print(f"\nPrimeras filas:")
df.head()

In [ ]:
# Crear una tarea de clasificación
task = TaskClassif(
    data=df,
    target='species',
    id='iris_classification'
)

# Explorar la tarea
print(f"Tipo de tarea: {task.task_type}")
print(f"Número de features: {len(task.feature_names)}")
print(f"Número de observaciones: {task.nrow}")
print(f"Clases: {task.class_names}")
print(f"Número de clases: {task.n_classes}")
print(f"\nFeatures disponibles:")
print(task.feature_names)

## 4. Crear un Learner

Utilizaremos learners de scikit-learn:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Crear un Random Forest learner
rf_learner = LearnerClassifSklearn(
    classifier="RandomForestClassifier",
    n_estimators=100,
    random_state=42,
    id='random_forest'
)

print(f"Learner ID: {rf_learner.id}")
print(f"¿Está entrenado?: {rf_learner.is_trained}")

## 5. Entrenar y Predecir

Veamos cómo entrenar el modelo y hacer predicciones:

In [ ]:
# Entrenar el learner
rf_learner.train(task)
print(f"¿Está entrenado ahora?: {rf_learner.is_trained}")

# Hacer predicciones
predictions = rf_learner.predict(task)

# Ver algunas predicciones
print("\nPrimeras 10 predicciones:")
print(predictions.response[:10])
print("\nClases reales:")
print(predictions.truth[:10])

## 6. Evaluar el Rendimiento

Usemos medidas para evaluar las predicciones:

In [ ]:
# Crear medidas
accuracy = MeasureClassifAccuracy()
f1 = MeasureClassifF1(average='macro')

# Calcular scores
acc_score = accuracy.score(predictions, task)
f1_score = f1.score(predictions, task)

print(f"Accuracy: {acc_score:.3f}")
print(f"F1-Score (macro): {f1_score:.3f}")

## 7. Cross-Validation

Para una evaluación más robusta, usemos cross-validation:

In [ ]:
# Definir estrategia de resampling
cv = ResamplingCV(
    folds=5,
    stratify=True  # Mantener proporción de clases
)

# Ejecutar evaluación con cross-validation
result = resample(
    task=task,
    learner=rf_learner,
    resampling=cv,
    measures=[accuracy, f1]
)

print(f"Número de iteraciones: {result.n_iters}")
print(f"Número de errores: {result.n_errors}")

# Ver resultados agregados
print("\nResultados agregados:")
print(result.aggregate())

# Obtener score promedio para accuracy
print(f"\nAccuracy promedio: {result.score('classif.acc', 'mean'):.4f}")
print(f"F1 promedio: {result.score('classif.f1', 'mean'):.4f}")

## 8. Comparar Múltiples Modelos

Creemos varios learners y comparemos su rendimiento:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Lista de learners a comparar
learners = [
    LearnerClassifSklearn(
        classifier="RandomForestClassifier",
        n_estimators=100,
        random_state=42,
        id='random_forest'
    ),
    LearnerClassifSklearn(
        classifier="DecisionTreeClassifier",
        max_depth=5,
        random_state=42,
        id='decision_tree'
    ),
    LearnerClassifSklearn(
        classifier="SVC",
        probability=True,
        random_state=42,
        id='svm'
    )
]

# Evaluar todos los learners
results = {}
for learner in learners:
    print(f"Evaluando {learner.id}...")
    result = resample(
        task=task,
        learner=learner,
        resampling=cv,
        measures=[accuracy]
    )
    results[learner.id] = result.score('classif.acc', 'mean')
    print(f"  Accuracy: {results[learner.id]:.4f}")

# Mostrar ranking
print("\nRanking de modelos:")
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
for i, (name, score) in enumerate(sorted_results, 1):
    print(f"{i}. {name}: {score:.4f}")

## 9. Usar SimpleAutoML

MLPY incluye una interfaz de AutoML simplificada:

In [ ]:
from mlpy.automl import SimpleAutoML

# Crear AutoML con configuración rápida
automl = SimpleAutoML(
    time_limit=60,  # 1 minuto
    max_models=5,   # Solo 5 modelos para demostración
    feature_engineering=True,
    feature_selection=True,
    cross_validation=3,  # 3 folds para rapidez
    test_size=0.2,
    random_state=42,
    verbose=True
)

# Ejecutar AutoML
automl_result = automl.fit(df, 'species')

print(f"\n🎯 Mejor score encontrado: {automl_result.best_score:.4f}")
print(f"⏱️ Tiempo total: {automl_result.training_time:.1f}s")
print(f"🔍 Modelos probados: {len(automl_result.leaderboard)}")

# Ver leaderboard
print("\n📊 Top 3 modelos:")
print(automl_result.leaderboard.head(3))

## 10. Crear un Pipeline

MLPY soporta pipelines para preprocesamiento:

In [ ]:
from mlpy.pipelines import Graph, GraphLearner, PipeOpScale, PipeOpLearner

# Crear pipeline: escalar → clasificar
pipeline_graph = Graph()

# Añadir operadores
scaler = PipeOpScale(id='scaler', method='standard')
classifier = PipeOpLearner(
    LearnerClassifSklearn(
        classifier="RandomForestClassifier",
        n_estimators=100,
        random_state=42
    ),
    id='classifier'
)

pipeline_graph.add_pipeop(scaler)
pipeline_graph.add_pipeop(classifier)
pipeline_graph.add_edge('scaler', 'output', 'classifier', 'input')

# Convertir a GraphLearner
pipeline = GraphLearner(graph=pipeline_graph, id='pipeline_rf')

# Evaluar el pipeline
pipeline_result = resample(
    task=task,
    learner=pipeline,
    resampling=cv,
    measures=[accuracy, f1]
)

print("Pipeline (Scale + RF) - Resultados:")
print(f"Accuracy: {pipeline_result.score('classif.acc', 'mean'):.4f}")
print(f"F1-Score: {pipeline_result.score('classif.f1', 'mean'):.4f}")

## 11. Visualización de Resultados

Visualicemos la comparación de modelos:

In [ ]:
import matplotlib.pyplot as plt

# Preparar datos para visualización
model_names = ['Random Forest', 'Decision Tree', 'SVM', 'AutoML', 'Pipeline']
accuracies = [
    results['random_forest'],
    results['decision_tree'],
    results['svm'],
    automl_result.best_score,
    pipeline_result.score('classif.acc', 'mean')
]

# Crear gráfico de barras
plt.figure(figsize=(12, 6))
bars = plt.bar(model_names, accuracies, 
               color=['lightblue', 'lightcoral', 'lightgreen', 'gold', 'orchid'])
plt.ylabel('Accuracy')
plt.title('Comparación de Modelos en Dataset Iris')
plt.ylim(0.8, 1.0)
plt.grid(axis='y', alpha=0.3)

# Añadir valores en las barras
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
             f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Mostrar el mejor modelo
best_idx = accuracies.index(max(accuracies))
print(f"\n🏆 Mejor modelo: {model_names[best_idx]} con accuracy de {max(accuracies):.4f}")

## 12. Próximos Pasos

¡Felicidades! Has aprendido los conceptos básicos de MLPY:

1. ✅ Crear tareas (Tasks)
2. ✅ Usar learners de scikit-learn
3. ✅ Evaluar modelos con measures
4. ✅ Usar cross-validation
5. ✅ Comparar múltiples modelos
6. ✅ Usar SimpleAutoML
7. ✅ Crear pipelines

### Para explorar más:

- **Notebooks avanzados**: Consulta otros notebooks para ejemplos más complejos
- **Documentación**: Lee la documentación completa en el README
- **Datasets grandes**: Prueba con backends Dask o Vaex
- **Tuning avanzado**: Optimización de hiperparámetros con Optuna
- **Visualización**: Más tipos de gráficos para análisis

¡MLPY está listo para proyectos de machine learning serios!